<a href="https://colab.research.google.com/github/Lakshya-mogha/Langgraph/blob/main/conditional%26looping_graphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install langgraph

In [29]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

In [30]:
class AgentState(TypedDict):
  num1:int
  operation:str
  num2:int
  result:int

In [31]:
def add(state:AgentState) -> AgentState:
  ''' add 2 numbers '''
  state['result'] = state['num1'] + state['num2']
  return state

def sub(state:AgentState) -> AgentState:
  ''' subtract 2 numbers '''
  state['result'] = state['num1'] - state['num2']
  return state

def decide_next_node (state:AgentState) -> AgentState:
  ''' select next node of the graph '''

  if state['operation'] == "+":
    return "add"
  elif state['operation'] == "-":
    return "sub"

In [32]:
graph = StateGraph(AgentState)

graph.add_node("add",add)
graph.add_node("sub",sub)
graph.add_node("router", lambda state:state) #passthrough func

graph.add_edge(START,"router")

graph.add_conditional_edges(
    "router",
    decide_next_node,
    {
        # Edge: Node
        "add": "add",
        "sub": "sub"
    }
)

graph.add_edge("add",END)
graph.add_edge("sub",END)

app = graph.compile()

In [33]:
initial_state_1 = AgentState(num1=10,operation="+", num2=5)
print(app.invoke(initial_state_1)['result'])

15


# Looping

In [34]:
import random
from typing import List,Dict

In [35]:
class AgentState(TypedDict):
  name:str
  number: List[int]
  count: int

In [36]:
def greeting(state:AgentState) -> AgentState:
  ''' greeting node  '''
  state["name"] = f"hi ,{state['name']}"
  state["count"] = 0

  return state

def gen_random(state:AgentState) -> AgentState:
  ''' generate random number '''
  state["number"].append(random.randint(0,10))
  state["count"] += 1

  return state

def should_continue(state:AgentState) -> AgentState:
  if state["count"] < 5:
    return "loop"
  else:
    return "exit"

In [37]:
graph = StateGraph(AgentState)

graph.add_node("greeting",greeting)
graph.add_node("random",gen_random)
graph.add_edge("greeting","random")

graph.add_conditional_edges(
    "random",
    should_continue,
    {
        "loop":"random",
        "exit": END
    }
)

graph.set_entry_point("greeting")

agent = graph.compile()


In [38]:
intial_state = AgentState(name="bob",number=[],count=-1)
print(agent.invoke(intial_state))

{'name': 'hi ,bob', 'number': [7, 1, 3, 0, 2], 'count': 5}


## There can be many other ways to make loops